In [1]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from google.cloud import storage
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Function to get data

def get_stock_data_from_gcp(nrows=10000, local=False, optimize=False, **kwargs):
    """method to get the training data (or a portion of it) from google cloud bucket"""
    # Add Client() here
    client = storage.Client()
    if local:
        path = "data/data_data_10Mill.csv"
    else:
        path = "gs://stock-news-pred-bucket/sorted_final_stock_prices"
    df = pd.read_csv(path)#add nrows after if you want to select a specific number of row
    return df

In [3]:
df = get_stock_data_from_gcp()

In [4]:
df.dropna(inplace=True)

In [5]:
df

,date,AAPL,MSFT,GOOGL,AMZN
1205,2004-08-19,30.71,27.12,100.335,38.63
1206,2004-08-20,30.80,27.20,108.310,39.51
1207,2004-08-23,31.08,27.24,109.400,39.45
1208,2004-08-24,31.95,27.24,104.870,39.05
1209,2004-08-25,33.05,27.55,106.000,40.30
...,...,...,...,...,...
5553,2021-11-24,161.94,337.91,2922.400,3580.41
5554,2021-11-26,156.81,329.68,2843.660,3504.56
5555,2021-11-29,160.24,336.63,2910.610,3561.57
5556,2021-11-30,165.30,330.59,2837.950,3507.07


In [6]:
# Function to scale data

def scale_stock_data(df, ticker):
    X = df[[ticker]]
    sc = MinMaxScaler().fit(X)
    X_sc = sc.fit_transform(X)
    return sc, X_sc

In [7]:
# Function to split scaled data

def split_stock_data(X_sc, train_size):
    index = round(train_size*X_sc.shape[0])
    X_sc_train = X_sc[:index]
    X_sc_test = X_sc[index:]
    return X_sc_train, X_sc_test

In [8]:
# Function to reshape scaled train data

def reshape_train_data(X_sc_train, obs):
    X_train = []
    y_train = []
    for i in range(obs, X_sc_train.shape[0]):
        X_train.append(X_sc_train[i-obs:i, 0])
        y_train.append(X_sc_train[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    return X_train, y_train

In [9]:
# Function to reshape scaled test data

def reshape_test_data(X_sc_test, obs):
    X_test = []
    y_test = []
    for i in range(obs, X_sc_test.shape[0]):
        X_test.append(X_sc_test[i-obs:i, 0])
        y_test.append(X_sc_test[i, 0])
    X_test, y_test = np.array(X_test), np.array(y_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    return X_test, y_test

In [10]:
# Function to inverse transform

def inverse_transformer(y):
    y = np.c_[y, np.ones(len(y))]
    y = scaler.inverse_transform(y)
    y = y[:,0]
    return y

In [11]:
# Function to build the lstm model

def lstm_model():
    model = Sequential()
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    model.add(Dense(units = 6))
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return model

In [12]:
# Function to train the model

def train_model(X_train, y_train):   
    model = lstm_model()
    es = EarlyStopping(patience=30, restore_best_weights=True)
    model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=[es], verbose=1)
    return model

In [13]:
# Function to predict results

def predict_model(ticker):
    sc, X_sc = scale_stock_data(df, ticker) 
    X_sc_train, X_sc_test = split_stock_data(X_sc, 0.8)
    X_train, y_train = reshape_train_data(X_sc_train, 30)
    X_test, y_test = reshape_test_data(X_sc_test, 30)
    #sc = MinMaxScaler().fit(X)
    model = train_model(X_train, y_train)
    predicted_stock_price = model.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    return predicted_stock_price

In [14]:
# New function to train model for then extracting predictions

def model_training(ticker):
    sc, X_sc = scale_stock_data(df, ticker) 
    X_sc_train, X_sc_test = split_stock_data(X_sc, 0.8)
    X_train, y_train = reshape_train_data(X_sc_train, 30)
    X_test, y_test = reshape_test_data(X_sc_test, 30)
    model = train_model(X_train, y_train)
    return model

In [15]:
# New reshape function for final predictions on X

def reshape_data(X_sc, obs):
    X_final = []
    for i in range(obs, X_sc.shape[0]):
        X_final.append(X_sc[i-obs:i, 0])
    X_final = np.array(X_final)
    X_final = np.reshape(X_final, (X_final.shape[0], X_final.shape[1], 1))
    return X_final

In [16]:
model_googl = model_training('GOOGL')

2021-12-02 12:00:05.980860: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-02 12:00:06.045671: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
87/87 [==============================] - 10s 50ms/step - loss: 0.0029 - val_loss: 3.3852e-04
Epoch 2/100
87/87 [==============================] - 3s 35ms/step - loss: 9.8689e-04 - val_loss: 2.1153e-04
Epoch 3/100
87/87 [==============================] - 3s 35ms/step - loss: 7.8139e-04 - val_loss: 1.6180e-04
Epoch 4/100
87/87 [==============================] - 4s 41ms/step - loss: 5.9282e-04 - val_loss: 6.8464e-04
Epoch 5/100
87/87 [==============================] - 4s 47ms/step - loss: 5.4562e-04 - val_loss: 2.3874e-04
Epoch 6/100
87/87 [==============================] - 4s 43ms/step - loss: 4.6087e-04 - val_loss: 1.1791e-04
Epoch 7/100
87/87 [==============================] - 4s 42ms/step - loss: 3.8987e-04 - val_loss: 3.7337e-04
Epoch 8/100
87/87 [==============================] - 4s 41ms/step - loss: 3.6297e-04 - val_loss: 2.1109e-04
Epoch 9/100
87/87 [==============================] - 4s 41ms/step - loss: 3.2995e-04 - val_loss: 3.5858e-04
Epoch 10/100
87/87 [===========

87/87 [==============================] - 3s 37ms/step - loss: 9.2822e-05 - val_loss: 3.1666e-05
Epoch 77/100
87/87 [==============================] - 3s 37ms/step - loss: 9.2152e-05 - val_loss: 2.1710e-05
Epoch 78/100
87/87 [==============================] - 3s 37ms/step - loss: 9.2427e-05 - val_loss: 3.1425e-05
Epoch 79/100
87/87 [==============================] - 3s 39ms/step - loss: 9.0920e-05 - val_loss: 2.9958e-05
Epoch 80/100
87/87 [==============================] - 3s 37ms/step - loss: 1.0321e-04 - val_loss: 1.2885e-04
Epoch 81/100
87/87 [==============================] - 3s 39ms/step - loss: 9.6918e-05 - val_loss: 1.8966e-04
Epoch 82/100
87/87 [==============================] - 3s 39ms/step - loss: 1.1414e-04 - val_loss: 9.2519e-05
Epoch 83/100
87/87 [==============================] - 4s 44ms/step - loss: 1.0761e-04 - val_loss: 1.7652e-04
Epoch 84/100
87/87 [==============================] - 3s 36ms/step - loss: 1.0622e-04 - val_loss: 1.5281e-04
Epoch 85/100
87/87 [============

In [17]:
sc, X_sc = scale_stock_data(df, 'GOOGL')

In [18]:
X_sc.shape

(4353, 1)

In [19]:
X_final = reshape_data(X_sc, 30)

In [20]:
X_final.shape

(4323, 30, 1)

In [21]:
final_predictions = model_googl.predict(X_final)

In [22]:
final_predictions

array([[0.0200457 , 0.01985699, 0.02026343, 0.02010201, 0.02029669,
        0.0201223 ],
       [0.02060214, 0.02041307, 0.0208185 , 0.02065696, 0.02085331,
        0.02067769],
       [0.0213418 , 0.02115327, 0.02155599, 0.02139486, 0.02158992,
        0.02141672],
       ...,
       [0.83766747, 0.8384713 , 0.8377023 , 0.8381388 , 0.8366147 ,
        0.8363728 ],
       [0.875515  , 0.87632143, 0.8755821 , 0.8758162 , 0.87382007,
        0.87438416],
       [0.8315153 , 0.83232844, 0.8315904 , 0.8319944 , 0.83047783,
        0.8302368 ]], dtype=float32)

In [23]:
final_predictions = sc.inverse_transform(final_predictions)
final_predictions

array([[ 158.07758,  157.53093,  158.7083 ,  158.24069,  158.80464,
         158.29948],
       [ 159.68945,  159.14177,  160.31621,  159.84825,  160.41704,
         159.9083 ],
       [ 161.83208,  161.28595,  162.45253,  161.9858 ,  162.55081,
         162.0491 ],
       ...,
       [2526.5317 , 2528.86   , 2526.6326 , 2527.897  , 2523.4822 ,
        2522.7812 ],
       [2636.1667 , 2638.503  , 2636.3613 , 2637.0396 , 2631.257  ,
        2632.891  ],
       [2508.7104 , 2511.0657 , 2508.928  , 2510.0981 , 2505.705  ,
        2505.0068 ]], dtype=float32)

In [24]:
final_predictions.shape

(4323, 6)

In [25]:
googl_predictions = pd.DataFrame(final_predictions)
googl_predictions

,0,1,2,3,4,5
0,158.077576,157.530930,158.708298,158.240692,158.804642,158.299484
1,159.689453,159.141769,160.316208,159.848251,160.417038,159.908295
2,161.832077,161.285950,162.452530,161.985794,162.550812,162.049103
3,164.650543,164.107651,165.262054,164.796692,165.351151,164.867279
4,165.522522,164.976227,166.131744,165.667206,166.236389,165.733810
...,...,...,...,...,...,...
4318,2617.738525,2619.960693,2617.832275,2618.840576,2613.557373,2614.198242
4319,2627.285645,2629.488281,2627.499023,2628.359131,2622.930664,2623.786865
4320,2526.531738,2528.860107,2526.632568,2527.896973,2523.482178,2522.781250
4321,2636.166748,2638.502930,2636.361328,2637.039551,2631.257080,2632.891113


In [26]:
df_googl = df.drop(columns=['AMZN','AAPL','MSFT'])
df_googl

,date,GOOGL
1205,2004-08-19,100.335
1206,2004-08-20,108.310
1207,2004-08-23,109.400
1208,2004-08-24,104.870
1209,2004-08-25,106.000
...,...,...
5553,2021-11-24,2922.400
5554,2021-11-26,2843.660
5555,2021-11-29,2910.610
5556,2021-11-30,2837.950


In [27]:
df_googl = df_googl.drop(df.head(30).index)
df_googl

,date,GOOGL
1235,2004-10-01,132.58
1236,2004-10-04,135.06
1237,2004-10-05,138.37
1238,2004-10-06,137.08
1239,2004-10-07,138.85
...,...,...
5553,2021-11-24,2922.40
5554,2021-11-26,2843.66
5555,2021-11-29,2910.61
5556,2021-11-30,2837.95


In [28]:
df_googl[['day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6']] = final_predictions

In [29]:
df_googl

,date,GOOGL,day_1,day_2,day_3,day_4,day_5,day_6
1235,2004-10-01,132.58,158.077576,157.530930,158.708298,158.240692,158.804642,158.299484
1236,2004-10-04,135.06,159.689453,159.141769,160.316208,159.848251,160.417038,159.908295
1237,2004-10-05,138.37,161.832077,161.285950,162.452530,161.985794,162.550812,162.049103
1238,2004-10-06,137.08,164.650543,164.107651,165.262054,164.796692,165.351151,164.867279
1239,2004-10-07,138.85,165.522522,164.976227,166.131744,165.667206,166.236389,165.733810
...,...,...,...,...,...,...,...,...
5553,2021-11-24,2922.40,2617.738525,2619.960693,2617.832275,2618.840576,2613.557373,2614.198242
5554,2021-11-26,2843.66,2627.285645,2629.488281,2627.499023,2628.359131,2622.930664,2623.786865
5555,2021-11-29,2910.61,2526.531738,2528.860107,2526.632568,2527.896973,2523.482178,2522.781250
5556,2021-11-30,2837.95,2636.166748,2638.502930,2636.361328,2637.039551,2631.257080,2632.891113


In [40]:
df_googl.to_csv('googl_predictions.csv', index=False)

In [31]:
#plt.figure(figsize=(20, 6))
#plt.plot(df['AMZN'], color = 'black', label = 'AMZN Stock Price')
#plt.plot(df['predictions'], color = 'green', label = 'Predicted AMZN Stock Price')
#plt.title('AMZN Stock Price Prediction')
#plt.xlabel('Time')
#plt.ylabel('AMZN Stock Price')
#plt.legend()
#plt.show()

In [32]:
#from keras.models import load_model

#model.save('stock_price.h5')

In [33]:
#model_test = load_model('lstm_model_stock_price.h5')

In [34]:
#sc, X_sc = scale_stock_data(df, 'AMZN')

In [35]:
#X_sc_train, X_sc_test = split_stock_data(X_sc, 0.8)

In [36]:
#X_train, y_train = reshape_train_data(X_sc_train, 30)

In [37]:
#X_test, y_test = reshape_test_data(X_sc_test, 30)

In [38]:
#y_pred = model_test.predict(X_test)

In [39]:
#y_pred